# Multimedia Clustering

## Loading data


For this assignment, we will use the [Paris Dataset](https://www.robots.ox.ac.uk/~vgg/data/parisbuildings/). This dataset comprises 6412 images taken at 12 well-known landmarks in the City of Love. These images were collected by querying the name for each landmark on [Flickr](https://www.flickr.com/).

Since downloading and clustering on the original set of images was quite slow, we decided to subsample this set. We uploaded this subsample, which contains 50 images per landmark, to Google Drive as a .zip file. By running the next cell, you can download and unzip this file:

In [ ]:
import gdown, zipfile, os, shutil

# The file id on the Drive
id = '1YbI4E82Ag_yY6Z2cM1x9QmCX8QUV0X8X'
url='https://drive.google.com/uc?id={}'.format(id)

# The output zip file
output = 'paris_sample.zip'
# Now download the file
gdown.download(url,output,quiet=False)
# Unzip the files
#os.path.abspath(output)
with zipfile.ZipFile(os.path.abspath(output), 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('paris')

Now we have a folder called 'Paris'! 
We can proceed to perfoming our initial loading of the images in that folder. Run the following three code cells:

In [ ]:
def get_all_filepaths(directory):
  '''
  A helper function to get all absolute file paths in a directory (recursively)
  :param directory:  The directory for which we want to get all file paths
  :return         :  A list of all absolute file paths as strings
  '''
  for dirpath,_,filenames in os.walk(directory):
    for f in sorted(filenames):
      yield os.path.abspath(os.path.join(dirpath, f))

In [ ]:
import cv2
from PIL import Image

def load_sample_images(sample_pathnames):
  '''
  Initial loading the given images. 
  :param sample_pathnames: An array of image file paths that need to be opened
  :return:                 A dictionary of the form key:image_dictionary,
                           whereby image_dictionary itself is a dictionary 
                           containing the original image and the preprocessed
                           image for later steps
  '''
  sample_images = {}
  for filename in sample_pathnames:  # Loop through all images, load each file
    sample_images[filename] = {}
    sample_images[filename]['original'] = Image.open(filename)
    sample_images[filename]['cv2'] = cv2.imread(filename)
  return(sample_images)     

In [ ]:
sample_pathnames = sorted(list(get_all_filepaths('paris')))
sample_images = load_sample_images(sample_pathnames)
print('We have loaded ' + str(len(sample_pathnames)) + ' images!')

An explanation of how you can access information about loaded images: 

***sample_pathnames*** is a list that contains all absolute file paths as strings. Filenames can be accessed by an index (a value between 0 and 599 since we have 600 images)

***sample_images*** is a dictionary. The sample images' file paths are the keys for that dictionary. This means that you can access the information of a specific image by using that image's file path as a key. What you will then get is again a dictionary. At this point in the assignment, this dictionary has two possible keys, '*original*' and '*cv2*'. 
*   The key '*original*' is the string that accesses the image loaded using the PIL library. 
*   The key '*cv2*' is the string that accesses the image loaded using the cv2 library. We will use this second format for one type of image feature representation below.

Please find usage examples below:

In [ ]:
# Get filename of the image with index 2
filename = sample_pathnames[2]
print('This is our filename: {}'.format(filename))

# Get the dictionary for that image
print('This is the dictionary for this image:')
print(sample_images[filename])

# Get the image loaded using PIL:
print('The image when loaded using PIL:')
print(sample_images[filename]['original'])

# Get the image loaded using cv2:
print('The image when loaded using cv2:')
print(sample_images[filename]['cv2'])

### First impression

Let's take a look at each landmark's first image to get a feeling for our dataset.
Please note: To access the information we loaded for each image, we have to use the full path name of that image as a key in our sample_images dictionary.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline 
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 12)) 
for i in range(0,12): 
  subplot = fig.add_subplot(4,3,i+1)
  imgplot = plt.imshow(sample_images[sample_pathnames[i*50]]['original'])
  subplot.set_title(sample_pathnames[i*50].split('/')[-2])
fig.tight_layout()
plt.show()

## Clustering on simple features


The images are loaded and we can start extracting some interesting properties from them to use as feature representations. 
In this subsection, we start with simple, straight-forward image features. The function *create_features_all_samples* below will be used from now on to create feature vectors for all images at once. 

In [ ]:
import time

def create_features(feature_funcs, image_dict):
  '''
  Loops over function names and calls each function and applies them on image
  :param feature_funcs:   A list of functions that extract individual features
  :param image_dict:       The loaded image dictionary to extract features from
  :return:                 The feature vector for the image
  '''
  feature_vector = []  
  for func in feature_funcs: # Subsequently apply each function
    feature_vector.extend(func(image_dict)) 
  return feature_vector  


def create_features_all_samples(feature_funcs,verbose=False):
  '''
  Loops over all sample images and calls create_features for each of them.
  :param feature_funcs:   A list of functions that extract individual features 
  :param verbose:         True if print statements of progress are desired
  :return:                All feature vectors combined in an array
  '''
  feature_vectors = []
  start = time.time()
  counter = 0 # Used to compute progress
  for sample_pathname in sample_pathnames:
    feature_vectors.append(create_features(feature_funcs, sample_images[sample_pathname]))
    counter = counter +1
    if verbose:
      if counter%25 == 0:
        print('Number of samples with features extracted: {}'.format(counter))  
        print("Time elapsed for last 25 samples: {}".format(time.time() - start))
        start = time.time()
  return feature_vectors

### Simple feature 1: Average color

The first feature we will extract is an image's average color.

Inspecting the format of an image loaded using PIL, you  already saw that the image was an **RGB** image. 
In the RGB format, a color value is represented by the intenisty of three colors: red **(R)**, green **(G)**, and blue **(B)**. Intensities are values between 0 and 255. 

The cell below shows nine example RGB color values:


In [ ]:
fig, axs = plt.subplots(3, 3)
fig.tight_layout()

# Add 9 example colors
axs[0, 0].imshow([[(255, 0, 0)]])
axs[0, 0].set_title('(255,   0,   0)')
axs[0, 1].imshow([[(0, 255,   0)]])
axs[0, 1].set_title('(  0,   255,   0)')
axs[0, 2].imshow([[(0, 0, 255)]])
axs[0, 2].set_title('(  0,   0, 255)')

axs[1, 0].imshow([[(0, 0, 0)]])
axs[1, 0].set_title('(  0,   0,   0)')
axs[1, 1].imshow([[(127, 127, 127)]])
axs[1, 1].set_title('(127,   127,   127)')
axs[1, 2].imshow([[(255, 255, 255)]])
axs[1, 2].set_title('(255, 255, 255)')

axs[2, 0].imshow([[(255, 0, 255)]])
axs[2, 0].set_title('(255,   0, 255)')
axs[2, 1].imshow([[(255, 255, 0)]])
axs[2, 1].set_title('(255,   255, 0)')
axs[2, 2].imshow([[(240, 128, 128)]])
axs[2, 2].set_title('(240,   128, 128)')


# Remove the ticks 
for row in axs:
  for ax in row:
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

In an **RGB image**, every pixel's color is represented by an RGB color value. 
An RGB image has three **color channels** (a red, a green and a blue channel). A color channel contains all pixels' intensity valuea for that color. For example, the red color channel contains all pixels' red intensity values (i.e. a number between 0 and 255 for each pixel). 

#### Task 1: Implement the function *average_color* in the cell below using the information provided in the comments. 
Hint: Remember that the original image can be accessed via *image_dict['original']*, and that you can convert this original image to a numpy array. (As a sanity check, make sure that you understand why the vector representation consists of three components in this case.)

Don't forget to copy your function to the answer file!

In [ ]:
import numpy as np

def average_color(image_dict):
  '''
  A function to compute the average RGB value of an image.
  First, average over rows to obtain an average value per column.
  Then, average over the resulting values to obtain one average value per color 
  channel.
  
  :param image_dict:  The dictionary containing the loaded image 
  :return:            A 3-dimensional np array: 1 average per color channel
  '''
  img = image_dict['original']
  np_img = np.array(img)
  rows,columns,channels =np_img.shape
  red = 0
  blue = 0
  green = 0
  pix_total = rows*columns
  for i in range(rows):
      for j in range(columns):
          k = np_img[i,j]
          if k[0] > k[1] and k[0] > k[2]:
            red = red + 1
            continue
          if k[1] > k[0] and k[1] > k[2]:
            green = green + 1
            continue
          if k[2] > k[0] and k[2] > k[1]:
            blue = blue + 1 
  res = np.array([red/pix_total,green/pix_total,blue/pix_total])
  print(res)
  return res
  


Let's see what the result of your function looks like. 
We can use an arbitrary sample image, e.g. the image with index 102:

In [ ]:
# Extract the average color
test_image_name = sample_pathnames[555] # Change the index here to see more examples
test_image_dict = sample_images[test_image_name] 
test_image = test_image_dict['original']
avg_color = average_color(test_image_dict).astype(int) # Cast for plotting

# Plot the test image next to the average color
fig = plt.figure(figsize=(12, 12))
plt.subplot(121), plt.imshow(test_image)
plt.title('Image: {}'.format(test_image_name)), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(Image.new("RGB", (test_image.size[0],test_image.size[1]),\
                  (avg_color[0], avg_color[1], avg_color[2])))
plt.title('The average color'), plt.xticks([]), plt.yticks([])
fig.show()

#### **Clustering on average color**

Now that we can successfully extract our first feature from images, it's time to perform some actual clustering on our dataset. 
Let's create a small feature vector for each sample image by extracting the average color:

In [ ]:
avg_color_feature_vectors = np.asarray([create_features([average_color], sample_images[x]) for x in sample_pathnames])
print('The dimensions of our array: {}'.format(avg_color_feature_vectors.shape))

As you can see, we have 600 images, each represented by a vector containing 3 components.

The function perform_k_means_clustering is where the clustering magic will happen. Run the two cells below to perform our first round of clustering (with 6 clusters) on the data using the small feature vectors:

In [ ]:
from sklearn.cluster import KMeans

def perform_k_means_clustering(feature_vectors, n_clusters=6):
  '''
  This function performs the clustering for us! It returns the n clusters.
  :param feature_vectors: The feature vectors that represent our data samples.
  :param n_clusters:      The number of clusters we want at the end (i.e. k)
  '''
  kmeans = KMeans(n_clusters=n_clusters,random_state=42).fit(feature_vectors)
  y_kmeans = kmeans.predict(feature_vectors)
  return y_kmeans

In [ ]:
# Hint: In the function above, n_clusters is set to 6 by default. 
y_kmeans_avg_color = perform_k_means_clustering(avg_color_feature_vectors) 
print("This is the clustering algorithm's output: ")
print(y_kmeans_avg_color)

As you can see, the function outputs a list that assigns each image to one of  the clusters. The indices between 0 and 5 each stand for one of the 6 clusters. We can illustrate this clustering outcome utilizing an interactive 3d scatter plot. Run the following two cells to create this plot:

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

def print_interactive_3d_cluster(features, feature_names, y_kmeans, filenames):
  '''
  A function to create an interactive (i.e. turnable) 3d scatterplot from our features.
  Importantly, this can only be done successfully if we only have 3 dimensions. 
  :param features:       A 3d vector 
  :param feature_names:  The names of the 3 features
  :param y_kmeans:       Our clustering algorithm's output (a list of integers)
  :param filenames:      Our images' filenames
  '''
  if not (features.shape[1]<3 or len(feature_names)!=3): # Check if dimensions fit
    # The following lines are a hack since plotly needs a pandas df as input
    helper = pd.DataFrame()
    helper[feature_names[0]] = features[:,0]
    helper[feature_names[1]] = features[:,1]
    helper[feature_names[2]] = features[:,2]
    helper['cluster_index'] = [str(clusternum) for clusternum in y_kmeans]
    helper['filenames'] = filenames
    # Create our scatter plot
    fig = px.scatter_3d(helper, x=feature_names[0], y=feature_names[1], z=feature_names[2], 
                       color = 'cluster_index', hover_data = ['filenames'])  
    fig.show()
  else:     
    print('Sorry. This function only works for 3-dimensional feature vectors..')

In [ ]:
fig = print_interactive_3d_cluster(avg_color_feature_vectors,['red_average','green_average','blue_average'], y_kmeans_avg_color,sample_pathnames)

However, we also want to see which images are in those clusters. Running the next two lines will enable you to automatically create and display an image montage for each cluster. This montage contains the images that are in that cluster.

In [ ]:
# You might have to restart your runtime after this line executes
!pip install --upgrade imutils

In [ ]:
from imutils import build_montages
from google.colab.patches import cv2_imshow 

def show_images_in_clusters(y_kmeans, sample_pathnames, sample_images):
  '''
  Create a montage for each cluster in y_kmeans featuring images in that cluster
  and subsequently display those montages. 
  :param y_kmeans:          The output vector of a clustering algorithm containing 
                            cluster indices for each sample
  :param sample_pathnames:  Our filenames list
  :param sample_images:     Our image dictionary 
  '''
  colnum = 9 # Specified by us, 9 columns looked the nicest.
  for cluster_index in np.unique(y_kmeans):
    montage_images = [np.asarray(sample_images[sample_pathnames[index]]['original']) \
                      for index, value in enumerate(y_kmeans) if value == cluster_index]
    rownum = int(len(montage_images)/colnum) # Calculate the number of rows we'll need
    if rownum == 0: rownum =1
    montages = build_montages(montage_images, (128, 196), (colnum, rownum))[0]
    fig = plt.figure(figsize=(30, 30))
    plt.title('Cluster index: {}'.format(cluster_index)), plt.xticks([]), plt.yticks([])
    plt.imshow(montages)

It's time to take a look at the images in our first clusters! 

In [ ]:
show_images_in_clusters(y_kmeans_avg_color, sample_pathnames, sample_images)

Hint: As it is a bit annoying to scroll within a notebook, we would like to point you to the full screen viewing mode. You can find it by clicking on the top right corner of the cell with the image output (three dots -> view output fullscreen).

### Simple feature 2: Color histograms

Instead of having one average value per color channel, so-called color histograms are a commonly used image feature. Let's take some minutes to understand the underlying idea behind this feature. For this purpose, we will zoom into an example image and investigate a tiny 15x15 pixel subarea:  




In [ ]:
test_image = Image.open('/content/paris/louvre/paris_louvre_000014.jpg')
# Crop the image -> zoomed-in view on lower right pixel values
w, h = test_image.size
subpart_test_image = np.array(test_image.crop((w-15, h-15, w, h)))

# Plot the image and the cropped subpart of the image
fig = plt.figure(figsize=(12, 12))
plt.subplot(121), plt.imshow(test_image)
plt.title('Our original image')
plt.subplot(122), plt.imshow(subpart_test_image)
plt.title('Cropped to 15x15 pixels')
fig.show()

As said above, we don't just want one average value for each color channel. Instead, we assign each pixel value of a color channel to a bin. Binning means that we group together values that fall within a certain interval. In our case, each bin represents a particular range of pixel values. Here, we use three bins. If a pixel's value falls within the first third of possible values, it is assigned to the first bin (index 0). If it is within the second third, it is assigned to the second bin (index 1), and if it is within the final third it is assigned to the third bin (index 2).

In [ ]:
# Create one image for each color channel:
for i in range(3):
  channel_img = np.zeros(subpart_test_image.shape, dtype='uint8')
  channel_img[:,:,i] = subpart_test_image[:,:,i]
  # First plot the color channel with the actual pixel values
  px_values = channel_img[:,:,i]
  plt.figure(figsize=(14,14))
  ax = plt.subplot(1,2,1)
  im = ax.imshow(channel_img)
  # Put the pixel values as text on top of the image:
  for j in range(channel_img.shape[0]):
    for k in range(channel_img.shape[1]):
      text = ax.text(k, j, px_values[j, k], ha="center", va="center", color="w")
  plt.title('Pixel values')
  # Now plot the color channel with the bin values
  bin_values = np.floor_divide(px_values,255/3).astype(int)
  ax = plt.subplot(1,2,2)
  im = ax.imshow(channel_img)
  for j in range(channel_img.shape[0]):
    for k in range(channel_img.shape[1]):
      text = ax.text(k, j, bin_values[j, k], ha="center", va="center", color="w")
  plt.title('Bin values')
  

As you can see, each pixel has been assigned to one bin for every color channel. This leaves us with 3x3x3 = 27 possible combinations of the individual red, green and blue bin values: 

| R | G | B |
| --- | --- | --- |
| 0 | 0 | 0 |
| 0 | 0 | 1 |
| 0 | 0 | 2 |
| 1 | 0 | 0 |
| 1 | 0 | 1 |
| 1 | 0 | 2 |
| 2 | 0 | 0 |
| 2 | 0 | 1 |
| 2 | 0 | 2 |
| 0 | 1 | 0 |
| 0 | 1 | 1 |
| 0 | 1 | 2 |
| 1 | 1 | 0 |
| 1 | 1 | 1 |
| 1 | 1 | 2 |
| 2 | 1 | 0 |
| 2 | 1 | 1 |
| 2 | 1 | 2 |
| 0 | 2 | 0 |
| 0 | 2 | 1 |
| 0 | 2 | 2 |
| 1 | 2 | 0 |
| 1 | 2 | 1 |
| 1 | 2 | 2 |
| 2 | 2 | 0 |
| 2 | 2 | 1 |
| 2 | 2 | 2 |

Our histogram is built by counting the number of pixels that belong to each of these combinations. In practice, you then normalize these counts to account for different sizes of images. This yields a normalized color histogram feature vector with 27 elements.

The following function was taken from a nice [blogpost](https://gogul.dev/software/image-classification-python) and adjusted for our purpose. It computes exactly this feature vector for a given image:

In [ ]:
def color_histogram(image_dict):
  '''
  Compute the normalized color histogram binned into 3x3x3 bins from the HSV image. 
  :param image_dict:  The dictionary containing the loaded image 
  :return:            A 27-dimensional np array
  '''
  # extract a 3D color histogram from the RGB color space
  im = image_dict['cv2']
  hist = cv2.calcHist([im], [0, 1, 2], None, [3,3,3], [0, 256, 0, 256, 0, 256])
  # normalize the histogram
  hist = cv2.normalize(hist,hist)
  # return the flattened histogram as the feature vector
  return hist.flatten() 

Let's see what the output for one vector looks like:




In [ ]:
test_chist_vector = color_histogram(test_image_dict)
test_chist_vector

**Clustering on color histograms**

See? We have our 27 element feature vector. Now we can perform clustering on our dataset using color histograms as features:

In [ ]:
chist_feature_vectors =  create_features_all_samples([color_histogram],verbose=False)
y_kmeans_chist = perform_k_means_clustering(chist_feature_vectors) 

Unfortunately, we cannot use our 3d scatter plot anymore since we have a higher-dimensional feature vector. However, we can inspect our clusters using the montages: 

In [ ]:
show_images_in_clusters(y_kmeans_chist, sample_pathnames, sample_images)

In some cases, three bins for each color channel might not be enough to represent semantic content in a dataset. In practice, researchers use 32 bins or more, depending on the problem. 


#### Task 2: Change the color histogram function below so that it uses 32 bins per color channel. Then run the second cell and inspect the results.

In [ ]:
def color_histogram_32bins(image_dict):
  '''
  Compute the normalized color histogram binned into 32x32x32 bins from the RGB image. 
  :param image_dict:  The dictionary containing the loaded image 
  :return:            A 32768-dimensional np array
  '''
  # extract a 3D color histogram from the RGB color space
  im = image_dict['cv2']
  hist = cv2.calcHist([im], [0, 1, 2], None, [3,3,3], [0, 256, 0, 256, 0, 256])
  # normalize the histogram
  hist = cv2.normalize(hist,hist)
  # return the flattened histogram as the feature vector
  return hist.flatten() 

In [ ]:
chist_32bins_feature_vectors =  create_features_all_samples([color_histogram_32bins],verbose=False)
y_kmeans_chist_32bins = perform_k_means_clustering(chist_32bins_feature_vectors) 
show_images_in_clusters(y_kmeans_chist_32bins, sample_pathnames, sample_images)

Our clustering function creates 6 clusters by default. We now want to see what happens if we increase the number of clusters. 


#### Task 3: Perform clustering with 12 clusters on the *chist_32bins_feature_vectors* and display the montages for these clusters in the cell below. 
Then repeat the same step with 24 clusters in the second cell below:

In [ ]:
y_kmeans_chist_12 = perform_k_means_clustering(chist_32bins_feature_vectors, n_clusters = 12)
show_images_in_clusters(y_kmeans_chist_12, sample_pathnames, sample_images)

In [ ]:
y_kmeans_chist_24 =perform_k_means_clustering(chist_32bins_feature_vectors, n_clusters = 24)
show_images_in_clusters(y_kmeans_chist_24, sample_pathnames, sample_images)

A side note:
In practice, instead of RGB, the HSV or CIELAB color spaces are mostly preferred. This is because they more closely reflect human color perception and the way we perceive differences between colors. With respect to the scope of this assignment, we decided to restrict ourselves to RGB here. Please be aware of the other color spaces in real-world projects. 

### Simple feature 3: Histogram of Oriented Gradients (HOG)

There is so much more to images than their colors!  Another simple but interesting image property is edges. Edges are typically characterized by a sudden dramatic change in color between pixels. Consider, for example, a black rectange on a white background. The rectangle's edges can be found by searching where pixel colors change from black to white. 

Here, we will be creating image representations based on edges. These involve using pixels' gradient vectors. Simply put, the gradient vector of each pixel in an image tells us how the color changes when moving from this pixel to its direct neighbour in a certain direction. This movement can happen in different directions: You can move left, right, up, down or along one of the four possible diagonals (upper/lower right and upper/lower left). These eight directions are known as oriented gradients. The histogram of these gradients tells us how strong these gradients are for a certain location in an image. Similarly to color histograms, HOGs use are created using binning (on orientations).

Let's take a look at our example image again!

In [ ]:
fig = plt.figure(figsize=(9,9))
plt.imshow(test_image) , plt.xticks([]), plt.yticks([])
plt.title('Test image')
plt.show()

We can visualize the test image's HOG by running the next cell. Luckily, there is a nice function called *hog* from the library *skimage* that can directly compute HOG for us.

In [ ]:
from skimage.feature import hog
from skimage import exposure

fd, hog_image = hog(test_image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True, feature_vector=True)
hog_image = exposure.rescale_intensity(hog_image, in_range=(0, 10))

fig = plt.figure(figsize=(9,9))
plt.imshow(hog_image, cmap=plt.cm.gray) , plt.xticks([]), plt.yticks([])
plt.title('HOG')
plt.show()

As you can see, the HOG is strongest at the edges in our test image. To perform clustering, we can now make a feature function using hog:

In [ ]:
def hog_vector(image_dict):
  '''
  Compute the histogram of oriented gradients binned into 8 bins. 
  :param image_dict:  The dictionary of loaded images for that image
  :return:            A feature vector of length 8        
  '''
  img = image_dict['original'].resize((300,300), Image.ANTIALIAS)  
  fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=True, feature_vector=True)  
  
  return fd

Let's see what the HOG representation for one image looks like:


In [ ]:
test_hog_vector = hog_vector(test_image_dict)
test_hog_vector

Our feature function only returns the feature representation (not the image we displayed above), which is a vector containing 8 elements (1 per orientation). 

**Clustering on HOG**

You might guess it already: It's time to perform our clustering! Since we considered 8 different directions of gradients, why don't we try producing 8 clusters?

In [ ]:
hog_feature_vectors =  create_features_all_samples([hog_vector],verbose=False)
y_kmeans_hog = perform_k_means_clustering(hog_feature_vectors, n_clusters=8)
show_images_in_clusters(y_kmeans_hog, sample_pathnames, sample_images)

## Clustering using neural representations

Neural networks can be used to create image representations that are capable of capturing image semantics more directly.

### Introducing neural representations

How this works: We use a neural network that has been trained to perform well on a huge image recognition task. In this assignment, the network we use is called 'VGG19'. The last layers in the network are responsible for classification, but other layers in the network can be used to extract feature representations. These feature representations are useful for clustering (and also classification, by the way) because they '...provide a high-level descriptor of the visual content of the image' ([Babenko et al. 2014](https://link.springer.com/chapter/10.1007/978-3-319-10590-1_38)).

Specifically, we can put our images into the VGG19 network and use the *fc2* layer as output layer.  With the command *summary*, we can inspect the architecture of the model at hand.  Let's take a look at the VGG19 network's original architecture:

In [ ]:
from keras.applications.vgg19 import VGG19
import tensorflow as tf
# from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array 
from keras.applications.vgg19 import preprocess_input
from keras.models import Model

original_model = VGG19(weights='imagenet')
original_model.summary()

We have to remove only VGG19's *predictions* layer to get the output from the *fc2* layer. Run the cell below to get rid of this layer.

In [ ]:
vgg19_without_last = Model(inputs=original_model.inputs, outputs=original_model.get_layer('fc2').output)

Our images need to be reshaped, since the model expects images of size (244, 244). We can store the reshaped images in our *sample_images* dictionary by running the two cells below:

In [ ]:
def load_image_to_format(image_path, format=(224,224)):
  '''
  Crop the image before resizing it. 
  :param image_path: The path to the image
  :return          : The resized image
  '''
  img = Image.open(image_path)
  height,width = img.size
  # fit in the biggest possible square to crop this
  square_size = min(height,width)
  # compute offsets
  x_offset = height - square_size
  y_offset = width - square_size
  img = img.crop((x_offset, y_offset,x_offset + square_size,y_offset + square_size))
  img = img.resize(format)
  return np.array(img)

In [ ]:
for filename in sample_pathnames:  
  img = load_image_to_format(filename) 
  preprocessed_img = img_to_array(img)
  preprocessed_img = np.expand_dims(preprocessed_img, axis=0)
  preprocessed_img = preprocess_input(preprocessed_img)   
  sample_images[filename]['preprocessed'] = preprocessed_img   

We can now use the following function to extract neural_representations from a given image file:

In [ ]:
def neural_representations(image_dict):
  '''
  A function that creates neural representations using the vgg19_without_last model.
  :param image:   The dictionary of the image
  :return:        A (flattened) numpy array containing the neural representations
  '''
  return np.array(vgg19_without_last.predict(image_dict['preprocessed'])).flatten()

Let's test this out for a random example image:



In [ ]:
print(neural_representations(sample_images[sample_pathnames[400]]).shape)

As you can see in the model's original architecture above, the *fc2* layer gives us 4096 features.  We can perform our clustering using those features. Please note that this might take a while. To get an estimation of the duration, we will enable the function argument *verbose*. 

In [ ]:
neural_representation_feature_vectors = create_features_all_samples([neural_representations],verbose=True)

We're there! Now we can finally conduct the clustering. We will tell the algorithm to create 12 clusters, since we have 12 different landmarks reflected in our dataset.

In [ ]:
y_kmeans_neural_representations = perform_k_means_clustering(neural_representation_feature_vectors,n_clusters=12) 
show_images_in_clusters(y_kmeans_neural_representations, sample_pathnames, sample_images)

#### Task 4: Now systematically vary the number of clusters in the cell below. Try out both more and fewer clusters. Then answer the questions below.

In [ ]:
y_kmeans_neural_representations_n = perform_k_means_clustering(neural_representation_feature_vectors,n_clusters=18)  # Vary the number here
show_images_in_clusters(y_kmeans_neural_representations_n, sample_pathnames, sample_images)

"Clustering bias" refers to the perspective that we decide to take on image similarity when we design and implement our clustering. The four image representations we investigated in our clustering study represent four different possible "clustering bias". 